## **探究对于标签偏移＋概念偏移的解决方法**
- 考虑迁移学习

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler

X = pd.read_csv("X_train.csv")
X_test2 = pd.read_csv("X_test_2.csv")
y_train = pd.read_csv("y_train.csv")["label"]
y_test2 = pd.read_csv("y_test_2_reduced.csv")["label"]

top_features = np.array([
    'F203', 'F18', 'F11', 'F58', 'F180', 'F137', 'F191', 'F273', 'F275',
    'F192', 'F188', 'F258', 'F281', 'F126', 'F225', 'F57', 'F13', 'F74',
    'F42', 'F1', 'F40', 'F288', 'F296', 'F128', 'F12', 'F104', 'F164',
    'F158', 'F229', 'F266', 'F289', 'F175', 'F152', 'F151', 'F32', 'F179',
    'F0', 'F165', 'F68', 'F244', 'F169', 'F66', 'F80', 'F2', 'F184', 'F45',
    'F291', 'F166', 'F298', 'F109'
])
top_feature_indices = [int(f[1:]) for f in top_features]


X_train_selected = X.iloc[:, top_feature_indices]
X_test2_selected = X_test2.iloc[:, top_feature_indices]


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test2_scaled = scaler.transform(X_test2_selected)


X_adapt_full_raw = X_test2_scaled[:202]
y_adapt_full_raw = y_test2[:202]


ros = RandomOverSampler(random_state=42)
X_adapt_full, y_adapt_full = ros.fit_resample(X_adapt_full_raw, y_adapt_full_raw)


X_adapt_train, X_eval, y_adapt_train, y_eval = train_test_split(
    X_adapt_full, y_adapt_full, test_size=0.45, stratify=y_adapt_full, random_state=42
)


X_joint = np.vstack([X_train_scaled, X_adapt_train])
y_joint = np.concatenate([y_train, y_adapt_train])


xgb = XGBClassifier(n_estimators=100, random_state=42)
xgb.fit(X_joint, y_joint)
xgb.save_model("xgb_final_model.json")


y_pred_eval = xgb.predict(X_eval)

f1_macro = f1_score(y_eval, y_pred_eval, average='macro')
f1_weighted = f1_score(y_eval, y_pred_eval, average='weighted')

print(f"Macro F1-score:     {f1_macro:.4f}")
print(f"Weighted F1-score:  {f1_weighted:.4f}")


Macro F1-score:     0.9216
Weighted F1-score:  0.9207


# 模型适配与性能恢复分析报告

## 一、背景与问题描述

在部署阶段，我们对训练好的客户反馈分类模型进行了测试。结果显示：

- 在新的测试集 `Test2` 上，模型性能显著下降；
- 微调前在 `Test2[:202]` 上的 F1-score 表现低下；
- 数据分析表明，`Test2` 的分布与训练集存在显著差异。

经分析判断，该现象源于数据分布发生了变化，即发生了如下两类偏移：

- **标签分布偏移（Label Shift）**：$P(y)_{\text{train}} \ne P(y)_{\text{test}}$
- **概念偏移（Concept Drift）**：$P(y \mid x)_{\text{train}} \ne P(y \mid x)_{\text{test}}$

此外，训练集和测试集中存在**严重的类别不均衡问题**，部分类别仅包含少量样本，进一步放大了偏移的影响。

---

## 二、目标与方案

本项目旨在：

1. 解决数据偏移与类别不均衡问题；
2. 设计一个兼顾泛化与适应性的模型微调流程；
3. 提升模型在 `Test2[:202]` 上的泛化性能，确保评估合理。

我们采用如下策略：

- 对训练集使用 `StandardScaler` 归一化并使用 `RandomOverSampler` 解决类别不平衡；
- 对 `Test2[:202]` 样本使用 `RandomOverSampler` 解决极少样本无法使用 SMOTE 的问题；
- 使用前 55% 样本进行微调，后 45% 样本进行验证，避免数据泄露；
- 使用 `XGBoost` 模型进行联合训练；
- 采用 `macro` 和 `weighted` F1-score 评估模型效果。

---

## 三、方法细节与公式推导

### 1. 标签偏移的建模与校正

标签偏移指 $P(y)$ 发生了变化，而 $P(x \mid y)$ 保持稳定。

为解决该问题，我们采用**重要性加权（Importance Weighting）**方法。令权重：

$$
w(y) = \frac{P_{\text{test}}(y)}{P_{\text{train}}(y)}
$$

训练时每个样本被赋予其标签对应的权重，从而优化如下目标：

$$
\min_{\theta} \sum_{i=1}^{n} w(y_i) \cdot \mathcal{L}(f_\theta(x_i), y_i)
$$

其中 $\mathcal{L}$ 为损失函数，$f_\theta$ 为待训练的模型。

> 由于 `Test2[:202]` 标签稀疏，我们未直接应用该策略在微调数据中，而是重点应用于训练集采样。

---

### 2. 类别不平衡的采样策略

在极度不平衡的情况下，少数类对整体损失贡献微弱，模型学习偏向多数类。

为此，我们使用 `RandomOverSampler` 进行采样，使每个类拥有相近样本数，从而缓解类别偏差问题。其目标是：

$$
\text{Balance:} \quad \forall y_k \in Y, \quad N_k \approx N_{\max}
$$

其中 $N_k$ 表示类别 $y_k$ 的样本数。

注意：我们未使用 SMOTE，因为其要求每类至少有两个样本，否则无法执行插值。

---

### 3. 微调策略设计

设微调数据为 $D_{\text{adapt}}$，我们将其划分为：

- 训练部分 $D_{\text{adapt}}^{\text{train}}$（55%）
- 验证部分 $D_{\text{adapt}}^{\text{val}}$（45%）

训练集联合为：

$$
D_{\text{joint}} = D_{\text{train}}^{\text{resampled}} \cup D_{\text{adapt}}^{\text{train}}
$$

训练目标为联合最小化：

$$
\min_{\theta} \sum_{(x_i, y_i) \in D_{\text{joint}}} w(y_i) \cdot \mathcal{L}(f_\theta(x_i), y_i)
$$

---

## 四、评估指标

采用两种 F1-score：

- **宏平均 F1-score（Macro F1）**：

  $$
  \text{F1}_{\text{macro}} = \frac{1}{K} \sum_{k=1}^{K} \text{F1}_k
  $$

  其中每个类的 F1 权重相等，更关注少数
